In [1]:
import xarray as xr 
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import h5py
from scipy.signal import detrend
from sklearn.decomposition import TruncatedSVD
import os
import geopandas as gpd
import regionmask
from scipy.stats import zscore
import itertools
import warnings
warnings.filterwarnings("ignore")
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from mpl_toolkits.axes_grid1 import make_axes_locatable
from joblib import Parallel, delayed
import math

In [2]:
def convert_to_non_nan_2d_array(ds):
# Check for valid time dimension
    if 'time' in ds.dims:
        time_len = len(ds.time.values)
    elif 'year' in ds.dims:  # Fallback to 'year' if 'time' is not available
        time_len = len(ds.year.values)
    else:
        raise ValueError("No valid time dimension found.")
    # Extract the dimensions for lat and lon
    lat_len = ds.sizes['lat']
    lon_len = ds.sizes['lon']
    
    # Reshape the data into a 2D array (time x space)
    scpdsi_2d = ds.values.reshape(time_len, lat_len * lon_len)
    
    # Identify columns that are not entirely NaN
    non_nan_mask = ~np.isnan(scpdsi_2d).any(axis=0)
    non_nan_mask[16719] =0  # these two numbers are where event series has all nan and they have to be removed manuallly
    non_nan_mask[137448] =0
    # Apply the mask to filter out columns with all NaN values
    scpdsi_non_nan = scpdsi_2d[:, non_nan_mask]
    
    return scpdsi_non_nan, non_nan_mask


In [3]:
filepath = "scPDSI.cru_ts4.06early1.1901.2021.cal_1950_21.bams.2022.GLOBAL.IGBP.WHC.1901.2021.nc"
scpdsi_nc = xr.open_dataset(filepath).sel(time = slice("1901","2021")).scpdsi.rename({"latitude":"lat","longitude":"lon"})
scpdsi_nc = scpdsi_nc.sel(time = slice("1901","2020"))
ds,mask = convert_to_non_nan_2d_array(scpdsi_nc)

In [4]:
lat = scpdsi_nc.lat.values
lon = scpdsi_nc.lon.values
coords = np.array(list(itertools.product(lon,lat)))
coords_valid = coords[mask]

In [5]:
def create_evt_series(array):
    b = array
    b = b.transpose(1,0)
    b = b[~np.isnan(b).all(axis=1)]
    b = np.less(b,-2)
    b1 = b[:,:1].astype(int)
    c = np.diff(b.astype(int))
    d = np.append(b1,c,axis=1)
    e = np.where(d == 1, 1, 0)
    e = e.astype(bool)
    f = np.arange(1, e.shape[1] + 1)
    g = np.where(e, f, False)
    # converting values to index
    non_zero_mask = (g != 0)
    result_series = np.where(non_zero_mask,g,np.nan)
    event_series = np.sort(result_series)
    return event_series

In [6]:
evt_final = create_evt_series(ds)
print(evt_final.shape)
evt_final = evt_final[~np.isnan(evt_final).all(axis=1),:]
print(evt_final.shape)

(59721, 1440)
(59721, 1440)


# Network Construction for time period 1901-2020

# preproceesing for creating event series

In [323]:
start_years_list = np.arange(1901,1962)
window_size = 60
def evt_series(start_year):
    new_dir = "/Event_series"
    end_year = start_year+window_size
    scpdsi_slice = scpdsi_nc.sel(time = (scpdsi_nc['time.year'] >=start_year) & (scpdsi_nc['time.year']<end_year))
    ds,_ = convert_to_non_nan_2d_array(scpdsi_slice)
    event_series = create_evt_series(ds)
    np.save(os.path.join(new_dir,f"event_series{start_year}.npy"), event_series)
with Parallel(n_jobs= 56,verbose = 1) as parallel:
    parallel(delayed(evt_series)(start_year) for start_year in start_years_list)



[Parallel(n_jobs=56)]: Using backend LokyBackend with 56 concurrent workers.
[Parallel(n_jobs=56)]: Done  12 out of  61 | elapsed:   14.7s remaining:  1.0min
[Parallel(n_jobs=56)]: Done  61 out of  61 | elapsed:   17.1s finished


In [173]:
import glob 
filepaths = glob.glob("/Event_series" +  '/' + '*npy')

In [ ]:
import re 
def tryint(s):
    try:
        return int(s)
    except ValueError:
        return s
    
def alphanum_key(s):
    """ Turn a string into a list of string and number chunks.
        "z23a" -> ["z", 23, "a"]
    """
    return [ tryint(c) for c in re.split('([0-9]+)', s) ]

def sort_nicely(l):
    """ Sort the given list in the way that humans expect.
    """
    l.sort(key=alphanum_key)
    sort_nicely(filepaths)
filepaths

In [ ]:
import os
import itertools
import time
from numba import jit
from joblib import Parallel, delayed

import numpy as np

import warnings
warnings.filterwarnings("ignore")

years = np.arange(1901,1962)
for i, file in enumerate(filepaths):

    events_load_path =  file
    number_chunks = 100000

    events_numpy = np.load(events_load_path)
    print(events_numpy.shape)

    # events_indices = np.load("All_Indices.npy")
    num_events = events_numpy.shape[0]
    num_pairs = num_events**2
    print(num_pairs)

    # Suppose num_pairs is square matrix, find number  of elements in upper triangle
    num_pairs_upper = int(num_pairs/2 + num_events/2) - num_events
    print(num_pairs_upper)

    # Get indices of upper triangle excluding diagonal
    events_indices = np.array(np.triu_indices(num_events, k=1)).reshape(2, -1).T
    print(events_indices.shape)

    @jit(nopython=True, nogil = True)
    def get_c_ij(ti, tj):
        ti = ti[~np.isnan(ti)]
        tj = tj[~np.isnan(tj)]
        si = len(ti)
        sj = len(tj)
        Cij = 0
        for l in range(0, si):
            tl_i = ti[l]
            time_diff = ti[l + 1] - tl_i if l < si - 1 else np.nan
            time_diff2 = tl_i - ti[l - 1] if l > 0 else np.nan
            for m in range(0, sj):
                tm_j = tj[m]
                time_diff3 = tj[m + 1] - tm_j if m < sj - 1 else np.nan
                time_diff4 = tm_j - tj[m - 1] if m > 0 else np.nan

                # t = np.abs(tl_i - tm_j)
                t = tl_i - tm_j

                time_lag = min(time_diff, time_diff2, time_diff3, time_diff4) / 2
                time_lag = time_lag if not np.isnan(time_lag) else 0

                if (0 < t) & (t < time_lag) & (time_lag <= 3):
                    j = 1
                elif t == 0:
                    j = 0.5
                else:
                    j = 0
                Cij += j
        return Cij

    @jit(nopython=True, nogil = True)
    def get_Q_ij(x, y):
        x = x[~np.isnan(x)]
        y = y[~np.isnan(y)]
        Si = len(x)
        Sj = len(y)
        if Si == 0 or Sj == 0:
            Qij = 0
        else:
            Qij = (get_c_ij(x, y) + get_c_ij(y, x)) / np.sqrt(Si * Sj)
        return Qij

    chunk_start_indices = np.linspace(0, events_indices.shape[0], number_chunks + 1).astype(int)

    # Number of events in each chunk
    chunk_size = chunk_start_indices[1] - chunk_start_indices[0]
    print(chunk_size)

    def get_Q_ij_chunk(events, start_idx, end_idx, events_indices):
        Q = [get_Q_ij(events[i, :], events[j, :]) for i,j in events_indices[start_idx:end_idx]]
        return Q

    start_time = time.time()
    with Parallel(n_jobs=48, verbose=1) as parallel:
        temp = parallel(delayed(get_Q_ij_chunk)(events_numpy, chunk_start_indices[idx], chunk_start_indices[idx + 1], events_indices) for idx in range(len(chunk_start_indices) - 1))

    end_time = time.time()

    print("Time taken: {} mins {} secs".format(int((end_time - start_time)/60), int((end_time - start_time)%60)))

    print("Saving results...")

    start_time = time.time()

    # Flatten list of lists
    temp = list(itertools.chain(*temp))

    # Convert list to numpy array
    temp = np.array(temp)

    # Use events_indices to put values in upper triangle
    Q = np.zeros((num_events, num_events))
    Q[events_indices[:, 0], events_indices[:, 1]] = temp
    Q = Q + Q.T

    end_time = time.time()
    print("Time taken: {} mins {} secs".format(int((end_time - start_time)/60), int((end_time - start_time)%60)))

    # Save Q
    Q_save_path = "Qij_matrix"
    Q_save_filename = os.path.join(Q_save_path, f"Q_{years[i]}.npy")
    start_time = time.time()
    np.save(Q_save_filename, Q)
    end_time = time.time()
    print("Time taken: {} mins {} secs".format(int((end_time - start_time)/60), int((end_time - start_time)%60)))

# Distance Calulation Matrix 


In [ ]:

number_chunks = 100000

events_numpy =  coords_valid
print(events_numpy.shape)

num_events = events_numpy.shape[0]
num_pairs = num_events**2
print(num_pairs)

num_pairs_upper = int(num_pairs/2 + num_events/2) - num_events
print(num_pairs_upper)

# Get indices of upper triangle excluding diagonal
events_indices = np.array(np.triu_indices(num_events, k=1)).reshape(2, -1).T
print(events_indices.shape)

@jit(nopython=True,nogil = True)
def haversine(l1, l2):
    lat1, lon1 = l1
    lat2, lon2 = l2
    lat1 = math.radians(lat1)
    lon1 = math.radians(lon1)
    lat2 = math.radians(lat2)
    lon2 = math.radians(lon2)
    earth_radius = 6371.0
    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    phi = (lat1+lat2)/2
    distance = earth_radius * math.sqrt((dlat**2) + (math.cos(phi)*dlon)**2)

    return distance


chunk_start_indices = np.linspace(0, events_indices.shape[0], number_chunks + 1).astype(int)

chunk_size = chunk_start_indices[1] - chunk_start_indices[0]
print(chunk_size)

def get_Q_ij_chunk(events, start_idx, end_idx, events_indices):
    Q = [haversine(events[i, :], events[j, :]) for i,j in events_indices[start_idx:end_idx]]
    return Q


start_time = time.time()
with Parallel(n_jobs=48, verbose=1) as parallel:
    temp = parallel(delayed(get_Q_ij_chunk)(events_numpy, chunk_start_indices[idx], chunk_start_indices[idx + 1], events_indices) for idx in range(len(chunk_start_indices) - 1))

end_time = time.time()



#  Flatten list of lists
temp = list(itertools.chain(*temp))

# Convert list to numpy array
temp = np.array(temp)

# Use events_indices to put values in upper triangle
D= np.zeros((num_events, num_events))
D[events_indices[:, 0], events_indices[:, 1]] = temp
D = D +D.T

end_time = time.time()
print("Time taken: {} mins {} secs".format(int((end_time - start_time)/60), int((end_time - start_time)%60)))

start_time = time.time()
D_com = D.astype(np.float32)
np.save("distance.npy", D_com)
end_time = time.time()
print("Time taken: {} mins {} secs".format(int((end_time - start_time)/60), int((end_time - start_time)%60)))